In [5]:
import librosa, glob, sys, json, pickle, os, random
from tqdm import tqdm
import pandas as pd, numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
accent_sizes_val = {'african':2000, 'indian':2500, 'hongkong':1200, 'philippines':2000,
                    'england':2500, 'scotland':2000, 'ireland':2000,
                    'canada':2000, 'us':2500}
accent_list = list(accent_sizes_val)
print("accents:{}".format(accent_list))

accents:['african', 'indian', 'hongkong', 'philippines', 'england', 'scotland', 'ireland', 'canada', 'us']


In [10]:
for accent in accent_sizes_val:
    os.system("tail -{} ../validated/accent-trans/{}.json > val/{}.json".format(accent_sizes_val[accent], accent, accent))

In [8]:
accent_sizes_inval = {'african':710, 'indian':2000, 'hongkong':479, 'philippines':355,
                    'england':2500, 'scotland':2000, 'ireland': 618,
                    'canada':2000, 'us':2500}
# print("accents:{}".format(accent_list))

accents:['african', 'indian', 'hongkong', 'philippines', 'england', 'scotland', 'ireland', 'canada', 'us']


In [11]:
for accent in accent_sizes_inval:
    os.system("tail -{} ../invalidated/accent-trans/{}.json > inval/{}.json".format(accent_sizes_inval[accent], accent, accent))

In [ ]:
accent_dirs = accents

list_ground = []
for accent in accent_dirs:
    path_selection = base_dir + accent + '/manifests/selection.json'
    list_selection = [json.loads(line.strip()) for line in open(path_selection)]
    list_ground.extend(list_selection)
    print(accent, len(list_selection))
print('ground', len(list_ground))

list_duration = []
for i in range(100):
    random.seed(i)
    samples = list_ground[:]
    random.shuffle(samples)
    duration = sum([item['duration'] for item in samples[:1000]])
    list_duration.append(duration)
print(f'mean {statistics.mean(list_duration)}, variance {statistics.variance(list_duration)}')

In [1]:
import IPython
IPython.display.Audio("./cv-corpus-7.0-2021-07-21/en/clips/common_voice_en_26349.mp3")


In [ ]:
json_folder = './trans/'
processed_json_folder = './processed_jsons/'
base_dir = './cv-corpus-7.0-2021-07-21/en/'
jsons = [f.name for f in os.scandir(json_folder) if 'json' in f.name]
# jsons = ['us.json']

def path_proc(pth):
    if './' in pth:
        return pth.replace('./', '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/mozilla/')
    return pth if '~/' not in pth else pth.replace('~/', '/home/mayank/') 

new_json_dir = ""
def convert(file_list, file_dir):
    with open(new_json_dir, 'w') as json_file:
        # with open(s, 'w') as json_file:
        for json_item in tqdm(file_list):
            mp3name = json_item['audio_filepath'].split('/en/wav/')[1].replace('wav', 'mp3')
            path_to_mp3 = base_dir+'clips/'+mp3name
            json_item['audio_filepath'] = path_proc(json_item['audio_filepath'])
            wav_file = base_dir + 'wav/' + mp3name.replace('.mp3', '.wav')
            if os.path.isfile(wav_file): os.unlink(wav_file)
            sound = AudioSegment.from_mp3(path_to_mp3)
            sound.export(wav_file, format="wav")
            duration = librosa.get_duration(filename=wav_file)
            json_item['duration'] = duration
            json.dump(json_item, json_file)
            json_file.write('\n')
        

for _dir in tqdm(jsons):
    json_path = json_folder + _dir
    new_json_dir = processed_json_folder + _dir
    print('_'*20)
    print(_dir)

    json_file = open(json_path)
    json_item_list = [line for line in json_file]
    json_item_list = [json.loads(line.strip()) for line in json_item_list]
    print('file_starting')
    print(json_path)
    convert(json_item_list, json_path)
    print(len(json_item_list))
    print('file_ending ...\n')